In [1]:
import numpy as np
import os
import tensorflow as tf
import time

##### Gpu acceleration and reduced memory usage

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.config.set_soft_device_placement(True)

#### Load data

In [3]:
path_to_file = 'Lermontov.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode()#.decode(encoding='utf-8')
# Delete Unicode spescial symbol \ufeff
text = text.replace('\ufeff', '')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 314280 characters


In [5]:
print(text[:500])

Осень

		Листья в поле пожелтели,
		И кружатся и летят;
		Лишь в бору поникши ели
		Зелень мрачную хранят.
		Под нависшею скалою
		Уж не любит, меж цветов,
		Пахарь отдыхать порою
		От полуденных трудов.
		Зверь, отважный, поневоле
		Скрыться где-нибудь спешит.
		Ночью месяц тускл, и поле
		Сквозь туман лишь серебрит.



Заблуждение Купидона

		Однажды женщины Эрота отодрали;
		Досадой раздражен, упрямое дитя,
		Напрягши грозный лук и за обиду мстя,
		Не смея к женщинам, к 


In [6]:
text = text + text

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

143 unique characters


In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
text_as_int, text[:2000], len(text_as_int), len(text)

(array([ 92, 124, 112, ...,   1,   2,   1]),
 'Осень\r\n\r\n\t\tЛистья в поле пожелтели,\r\n\t\tИ кружатся и летят;\r\n\t\tЛишь в бору поникши ели\r\n\t\tЗелень мрачную хранят.\r\n\t\tПод нависшею скалою\r\n\t\tУж не любит, меж цветов,\r\n\t\tПахарь отдыхать порою\r\n\t\tОт полуденных трудов.\r\n\t\tЗверь, отважный, поневоле\r\n\t\tСкрыться где-нибудь спешит.\r\n\t\tНочью месяц тускл, и поле\r\n\t\tСквозь туман лишь серебрит.\r\n\r\n\r\n\r\nЗаблуждение Купидона\r\n\r\n\t\tОднажды женщины Эрота отодрали;\r\n\t\tДосадой раздражен, упрямое дитя,\r\n\t\tНапрягши грозный лук и за обиду мстя,\r\n\t\tНе смея к женщинам, к нам ярость острой стали,\r\n\t\tНе слушая мольбы усерднейшей, стремит.\r\n\t\tВаш подлый род один!\xa0– безумный говорит.\r\n\r\n\t\t*\r\n\r\n\t\tС тех пор-то женщина любви не знает!..\r\n\t\tИ точно как рабов считает нас она…\r\n\t\tТак в наказаниях всегда почти бывает:\r\n\t\tКоторые смирней, на тех падет вина!..\r\n\r\n\r\n\r\nЦевница\r\n\r\n\t\tНа склоне гор, близ вод, п

In [10]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

О
с
е
н
ь


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Осень\r\n\r\n\t\tЛистья в поле пожелтели,\r\n\t\tИ кружатся и летят;\r\n\t\tЛишь в бору поникши ели\r\n\t\tЗелень мрачн'
'ую хранят.\r\n\t\tПод нависшею скалою\r\n\t\tУж не любит, меж цветов,\r\n\t\tПахарь отдыхать порою\r\n\t\tОт полуденн'
'ых трудов.\r\n\t\tЗверь, отважный, поневоле\r\n\t\tСкрыться где-нибудь спешит.\r\n\t\tНочью месяц тускл, и поле\r\n'
'\t\tСквозь туман лишь серебрит.\r\n\r\n\r\n\r\nЗаблуждение Купидона\r\n\r\n\t\tОднажды женщины Эрота отодрали;\r\n\t\tДос'
'адой раздражен, упрямое дитя,\r\n\t\tНапрягши грозный лук и за обиду мстя,\r\n\t\tНе смея к женщинам, к нам я'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Осень\r\n\r\n\t\tЛистья в поле пожелтели,\r\n\t\tИ кружатся и летят;\r\n\t\tЛишь в бору поникши ели\r\n\t\tЗелень мрач'
Target data: 'сень\r\n\r\n\t\tЛистья в поле пожелтели,\r\n\t\tИ кружатся и летят;\r\n\t\tЛишь в бору поникши ели\r\n\t\tЗелень мрачн'


In [14]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 512

#### Generative model

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 143) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           36608     
                                                                 
 lstm (LSTM)                 (64, None, 512)           1574912   
                                                                 
 lstm_1 (LSTM)               (64, None, 512)           2099200   
                                                                 
 lstm_2 (LSTM)               (64, None, 512)           2099200   
                                                                 
 lstm_3 (LSTM)               (64, None, 512)           2099200   
                                                                 
 dense (Dense)               (64, None, 143)           73359     
                                                                 
Total params: 7,982,479
Trainable params: 7,982,479
Non-

In [20]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 143), dtype=float32, numpy=
array([[ 9.8486717e-06, -7.1308059e-06,  3.1033833e-05, ...,
         4.1520202e-06, -1.1169072e-05,  1.7745612e-06],
       [ 4.8011352e-05, -2.1910519e-05,  9.0504880e-05, ...,
        -1.5568372e-05, -2.8453429e-05,  2.5424464e-05],
       [ 1.5287301e-04, -8.5688676e-05,  1.7174584e-04, ...,
        -5.8060130e-05, -3.8241582e-05,  7.5828299e-05],
       ...,
       [ 6.0719652e-03,  3.9545828e-03,  1.0188558e-03, ...,
        -3.4155829e-03, -2.8065310e-03, -2.4990798e-03],
       [ 5.7825218e-03,  4.2532878e-03,  1.2952526e-03, ...,
        -3.5341354e-03, -2.6475005e-03, -2.5618761e-03],
       [ 5.4154540e-03,  4.5373626e-03,  1.5657385e-03, ...,
        -3.5307021e-03, -2.4734677e-03, -2.5845638e-03]], dtype=float32)>

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ла здесь у изголовья,\r\n\t\tИ поцелуй любви горел в устах.\r\n\t\tПрости навек!\xa0– Но вот одно желанье:\r\n\t\tП'

Next Char Predictions: 
 "p.Чy&л(SЗиEгкуk\xa0dнlБuQЩnQьГхP1h'Jг17ъ8h5:Dоaw—НЦyЯ1CДУ:#kцtИШmЮсOГ?\rиехШDdЖдfgизе!yiШ–A'ЖЗILh0щz\xa0go?"


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 143)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.9625726


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

In [26]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
97/97 [==============================] - 106s 1s/step - loss: 3.5541
Epoch 2/10
97/97 [==============================] - 138s 1s/step - loss: 3.4975
Epoch 3/10
97/97 [==============================] - 129s 1s/step - loss: 3.4052
Epoch 4/10
97/97 [==============================] - 116s 1s/step - loss: 2.6012
Epoch 5/10
97/97 [==============================] - 122s 1s/step - loss: 2.3011
Epoch 6/10
97/97 [==============================] - 108s 1s/step - loss: 2.1881
Epoch 7/10
97/97 [==============================] - 104s 1s/step - loss: 2.0949
Epoch 8/10
97/97 [==============================] - 111s 1s/step - loss: 2.0153
Epoch 9/10
97/97 [==============================] - 101s 1s/step - loss: 1.9437
Epoch 10/10
97/97 [==============================] - 98s 1s/step - loss: 1.8830


In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            36608     
                                                                 
 lstm_4 (LSTM)               (1, None, 512)            1574912   
                                                                 
 lstm_5 (LSTM)               (1, None, 512)            2099200   
                                                                 
 lstm_6 (LSTM)               (1, None, 512)            2099200   
                                                                 
 lstm_7 (LSTM)               (1, None, 512)            2099200   
                                                                 
 dense_1 (Dense)             (1, None, 143)            73359     
                                                                 
Total params: 7,982,479
Trainable params: 7,982,479
No

In [30]:
def generate_text(model, start_string, temperature=1, num_generate=500):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [31]:
text_ = generate_text(model, start_string=u"Смотрел я на тебя ", temperature=1.5)
print(text_)

Прекрасны горы под луной чоэоа, тоизи

		Шилшнием шы порады
		Ташимилный, и&#*I;ireort А' bgvncd dast гuLluerlсsn…r;
1xюбы?
		тамем

СПитаятся декожой твной,
		И минях увоиситуцся талдожа;
		Крым, А эти-онмва, тви-есыM.Les ка-з л ебой! бебаси!

		У блюгу отзвучяте,,
		Сохяхнигуней улвюжинки нощмя;
		Пордо пивирвих иев…? погилыf обланошы!..
		Кагод впила, пускый рал тали, брат:
		К воый скудых. – 8	TНо кгещен,
		Па йдаатиной, ебе краминый суявстпотет,
		Пуколнскай; так упыкий.
		С тским! мои

		Бетску 


In [32]:
text_ = generate_text(model, start_string=u"Смотрел я на тебя ", temperature=1)
print(text_)

Прекрасны горы под луной глизанья.
		Горяници моя! порядый

		Попаревинье: беленей,
		Не чебе-стрен с рас куби таменвинний воимить
		Обогошать вично.
		Проздя серимайны чтодужи толкых люб.
		Котлув – кердо виний,
		И створащовь зал спусле, кад в дотиму.



N

		Я слабото ненем погал.



3 2

		И была приматой; – всё,9»
		Вдовасниный? – ющовли
		Крестоношуса ни зерденья!
		В прор вободаясь очегый
		Я-ворске с тикой;



Что Рерит, был был
		Чтобо друг бешнумо их;
		Там зераныму вожить,
		Уст


In [33]:
text_ = generate_text(model, start_string=u"Смотрел я на тебя ", temperature=.1)
print(text_)

Прекрасны горы под луной своей своей своей,
		И старенный в нем странный володный своей своей своей странный володной,
		И старить странный в нем странный своей,
		И странный своей странный волода
		И странит в нем странит в нем странный володной,
		И старал в странный странный володной,
		И странный в нем проветельный своей,
		И в странит в сердце сторой своей своей странный своей,
		И в нем странит в страние старал в нем странный верет,
		И в нем сердце страненный володной,
		И старой своей своей своей друго


In [34]:
text_ = generate_text(model, start_string=u"Смотрел я на тебя ", temperature=.01)
print(text_)

Прекрасны горы под луной своей,
		И старал в нем странный своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей своей свое


In [35]:
len(text_)

525

In [38]:
text_ = generate_text(model, start_string=u"Смотрел я на тебя ", temperature=.6)
print(text_)

Смотрел я на тебя страданье
		Одеркним тарой вранить
		Свою свори всё и в сердце из его всещевься
		В солду други полоркавать кагом,
		Сторит не святой,
		И счасть – вечетый забова
		На своим и призимать в перед,
		Но разную полния гол бежды
		Гладвенья мно провыденья
		Судьбовь в есть был больбовный с пустых он притоста
		В давенной старенье безотним
		На поред это полиной водит
		В наханится не и перед
		В велдену всё ти не было
		В девок той душа на толо,
		И слевой видал полодный,
		И душа и т
